In [9]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python:', python_version())

Versão da Linguagem Python: 3.10.9


In [2]:
# Imports
import pandas as pd
import numpy as np
import pymc3 as pm
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "José Guilherme Daufenbach Júnior" --iversions

Author: José Guilherme Daufenbach Júnior

pymc3 : 3.11.5
pandas: 2.0.2
numpy : 1.22.1



In [4]:
# Carregar o dataset de investimentos
dados_investimentos = pd.read_csv('dados/dados_investimentos.csv', index_col = 0, parse_dates = True)

In [5]:
dados_investimentos.shape


(365, 6)

In [6]:
dados_investimentos.head()

,Ações,Títulos Públicos,Fundos Imobiliários,Rendimento Ações,Rendimento Títulos Públicos,Rendimento Fundos Imobiliários
2022-01-01,-0.017334,-0.048899,0.510967,NaN,NaN,NaN
2022-01-02,0.435083,-0.159320,-0.189749,-26.099910,2.258122,-1.371353
2022-01-03,0.196220,-0.101227,0.159899,-0.549005,-0.364632,-1.842684
2022-01-04,0.023019,0.072812,-0.402519,-0.882688,-1.719296,-3.517339
2022-01-05,0.103289,0.136370,0.192497,3.487111,0.872901,-1.478231


In [7]:
dados = dados_investimentos[['Rendimento Fundos Imobiliários']]

In [8]:
dados.head()

,Rendimento Fundos Imobiliários
2022-01-01,NaN
2022-01-02,-1.371353
2022-01-03,-1.842684
2022-01-04,-3.517339
2022-01-05,-1.478231


In [11]:
deslocamento = 5
dados['Rendimento Fundos Imobiliários (t-5)'] = dados['Rendimento Fundos Imobiliários'].shift(deslocamento)

In [12]:
dados.head()

,Rendimento Fundos Imobiliários,Rendimento Fundos Imobiliários (t-5)
2022-01-01,NaN,NaN
2022-01-02,-1.371353,NaN
2022-01-03,-1.842684,NaN
2022-01-04,-3.517339,NaN
2022-01-05,-1.478231,NaN


In [13]:
dados.tail()

,Rendimento Fundos Imobiliários,Rendimento Fundos Imobiliários (t-5)
2022-12-27,-0.104117,-4.930165
2022-12-28,-1.485806,-1.328787
2022-12-29,0.113994,-1.616446
2022-12-30,-2.652999,-2.591367
2022-12-31,-1.680685,-3.610346


In [19]:
dados.rename(columns={'Rendimento Fundos Imobiliários': 'Rendimentos',
                      'Rendimento Fundos Imobiliários (t-5)': 'Rendimentos(t-5)'}, inplace=True)

In [20]:
dados.head()

,Rendimentos,Rendimentos(t-5)
2022-01-01,NaN,NaN
2022-01-02,-1.371353,NaN
2022-01-03,-1.842684,NaN
2022-01-04,-3.517339,NaN
2022-01-05,-1.478231,NaN


In [21]:
dados.dropna(inplace=True)

In [22]:
dados.head()

,Rendimentos,Rendimentos(t-5)
2022-01-07,-3.025357,-1.371353
2022-01-08,-1.948727,-1.842684
2022-01-09,-0.055173,-3.517339
2022-01-10,-1.490320,-1.478231
2022-01-11,-3.540331,-0.196477


In [23]:
#Divisão treino/teste
dados_treino = dados.loc['2022-01-06':'2022-10-31']
dados_teste = dados.loc['2022-11-01':'2022-12-31']

## *Modelo de regressão bayessiana:*

In [24]:
# Inicia um novo modelo pymc3:
with pm.Model() as modelo:
    
    #priors
    alpha = pm.Normal('alpha', mu = 0, sd = 1)
    beta = pm.Normal('beta', mu = 0, sd = 1)
    sigma = pm.HalfNormal('sigma', sd = 1)
    
    #likelihood
    mu = alpha + beta * dados_treino['Rendimentos(t-5)']
    likehood = pm.Normal('y', mu = mu, sd = sigma, observed = dados_treino['Rendimentos'])
    
    #Inferência
    trace = pm.sample(1000, tune = 1000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, beta, alpha]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 2 seconds.


In [25]:
#Criando preditor

rendimento_previsto = pm.sample_posterior_predictive(trace, 
                                                     samples = 1000, 
                                                     model = modelo,
                                                     var_names = ['alpha', 'beta', 'sigma', 'y'])['y'].mean(axis = 0)

In [27]:
rendimento_previsto

array([-1.70280495e-01,  8.95253564e-01,  6.79315474e-01,  2.25029494e-01,
        1.05896205e+00, -1.34538325e-01, -3.55845742e-01,  2.18099217e-01,
        1.93120597e-01,  8.71742002e-03, -4.04527704e-02,  4.45058023e-01,
        4.13063566e-02, -3.78801188e-01,  8.81746475e-01,  6.24086412e-01,
       -1.07182609e-01, -9.44023163e-01, -9.37154549e-01,  3.99257249e-01,
       -2.13389125e-01,  7.40601325e-01, -4.19354373e-01, -4.65998149e-01,
        1.40385577e-01,  5.64261462e-01,  3.92108822e-01,  8.34243812e-01,
        7.17994450e-01,  8.69283416e-01,  3.46379148e-01,  4.38159026e-01,
       -2.11477487e-02,  1.87382953e-01,  1.31606582e-01,  5.23998318e-02,
        2.50268275e-01,  5.57519153e-01,  5.06342937e-01,  3.42061809e-01,
       -4.35395632e-01, -9.47182300e-01, -1.74712600e-02,  2.55085110e-01,
        2.56372328e-01,  8.79586958e-01,  1.02362535e+00,  1.41100643e+00,
        5.18305213e-01, -3.04512701e-01,  5.07136398e-01,  4.85670765e-02,
        5.57186019e-01,  

In [29]:
n_dados = pd.read_csv('dados/novos_dados.csv', index_col = 0, parse_dates = True)

In [30]:
n_dados_preditor = n_dados['Rendimento Fundos Imobiliários'].shift(5)

In [31]:
n_dados_preditor = n_dados_preditor.dropna()

In [32]:
with modelo:
    alpha = trace['alpha'].mean()
    beta = trace['beta'].mean()
    
    previsoes = alpha + beta * n_dados_preditor

In [34]:
print(previsoes)

2023-01-07    0.243854
2023-01-08    0.246773
2023-01-09    0.249451
2023-01-10    0.370407
2023-01-11    0.244021
                ...   
2023-12-27    0.238750
2023-12-28    0.239093
2023-12-29    0.249185
2023-12-30    0.304548
2023-12-31    0.243404
Name: Rendimento Fundos Imobiliários, Length: 359, dtype: float64
